# 📤 Pinecone 벡터 DB 업로드 (3개 인덱스 분리)
## 주택임대차 RAG 시스템 - Upstage Embedding

**인덱스 구조:**
- `housing-law-index`: 법률 (주택임대차보호법, 민법)
- `housing-rule-index`: 시행령, 시행규칙
- `housing-case-index`: 판례, 상담사례

## 1️⃣ 환경 설정 및 라이브러리 임포트

In [44]:
# 패키지 설치 (최초 1회)
# !pip install python-dotenv pandas langchain langchain-upstage langchain-pinecone pinecone-client tqdm

In [45]:
import os
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

# LangChain
from langchain_core.documents import Document
from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

print("✅ 라이브러리 임포트 완료")

✅ 라이브러리 임포트 완료


## 2️⃣ 환경 변수 로드 및 경로 설정

In [46]:
# .env 파일 로드
load_dotenv()

# API 키
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")

# 인덱스명 (미리 Pinecone에서 생성해둔 인덱스)
LAW_INDEX_NAME = "housing-law-index"
RULE_INDEX_NAME = "housing-rule-index"
CASE_INDEX_NAME = "housing-case-index"

print("🔑 API 키 확인:")
print(f"  PINECONE_API_KEY: {'✅' if PINECONE_API_KEY else '❌ 없음'}")
print(f"  UPSTAGE_API_KEY: {'✅' if UPSTAGE_API_KEY else '❌ 없음'}")
print(f"\n📌 Pinecone 인덱스:")
print(f"  - Law: {LAW_INDEX_NAME}")
print(f"  - Rule: {RULE_INDEX_NAME}")
print(f"  - Case: {CASE_INDEX_NAME}")

🔑 API 키 확인:
  PINECONE_API_KEY: ✅
  UPSTAGE_API_KEY: ✅

📌 Pinecone 인덱스:
  - Law: housing-law-index
  - Rule: housing-rule-index
  - Case: housing-case-index


In [47]:
# Path 설정
BASE_DIR = Path.cwd().parents[1]  # 프로젝트 루트
CSV_DIR = BASE_DIR / "data" / "processed" / "csv"

LAW_CSV = CSV_DIR / "law.csv"
RULE_CSV = CSV_DIR / "rule.csv"
CASE_CSV = CSV_DIR / "case.csv"

print(f"📂 CSV 디렉토리: {CSV_DIR}")
print(f"   - law.csv 존재: {'✅' if LAW_CSV.exists() else '❌'}")
print(f"   - rule.csv 존재: {'✅' if RULE_CSV.exists() else '❌'}")
print(f"   - case.csv 존재: {'✅' if CASE_CSV.exists() else '❌'}")

📂 CSV 디렉토리: c:\project-AI2\실프로젝트\data\processed\csv
   - law.csv 존재: ✅
   - rule.csv 존재: ✅
   - case.csv 존재: ✅


## 3️⃣ Pinecone 클라이언트 초기화

In [48]:
# Pinecone 클라이언트 초기화
pc = Pinecone(api_key=PINECONE_API_KEY)

# 인덱스 존재 확인
existing_indexes = [idx.name for idx in pc.list_indexes()]

print("📊 Pinecone 인덱스 목록:")
for idx_name in existing_indexes:
    print(f"  ✅ {idx_name}")

# 필요한 인덱스 확인
required = [LAW_INDEX_NAME, RULE_INDEX_NAME, CASE_INDEX_NAME]
missing = [idx for idx in required if idx not in existing_indexes]

if missing:
    print(f"\n⚠️ 경고: 다음 인덱스가 없습니다: {missing}")
    print("   Pinecone 콘솔에서 미리 생성해주세요.")
else:
    print("\n✅ 모든 필요한 인덱스가 존재합니다.")

📊 Pinecone 인덱스 목록:
  ✅ real-estate-law-index
  ✅ housing-law-index
  ✅ housing-case-index
  ✅ housing-rule-index

✅ 모든 필요한 인덱스가 존재합니다.


## 4️⃣ Upstage 임베딩 모델 초기화

In [49]:
print("📦 Upstage 임베딩 모델 로드 중...")

embedding = UpstageEmbeddings(
    model="solar-embedding-1-large-passage",
    api_key=UPSTAGE_API_KEY
)

print("✅ 임베딩 모델 준비 완료")
print("   - Model: solar-embedding-1-large-passage")
print("   - Dimension: 4096")

📦 Upstage 임베딩 모델 로드 중...
✅ 임베딩 모델 준비 완료
   - Model: solar-embedding-1-large-passage
   - Dimension: 4096


In [50]:
# 임베딩 테스트
test_text = "보증금 반환을 안 해주면 어떻게 하나요?"
test_vector = embedding.embed_query(test_text)

print(f"🧪 임베딩 테스트:")
print(f"   문장: {test_text}")
print(f"   벡터 차원: {len(test_vector)}")
print(f"   벡터 샘플 (처음 5개): {test_vector[:5]}")

🧪 임베딩 테스트:
   문장: 보증금 반환을 안 해주면 어떻게 하나요?
   벡터 차원: 4096
   벡터 샘플 (처음 5개): [0.023319972679018974, -0.022519396618008614, 0.01442326046526432, 0.001789995119906962, -0.008767586201429367]


## 5️⃣ 헬퍼 함수 정의

In [ ]:
def load_csv_to_documents(csv_path):
    """
    CSV 파일을 로드하여 LangChain Document 리스트로 변환
    
    Args:
        csv_path: CSV 파일 경로
    
    Returns:
        List[Document]: LangChain Document 객체 리스트
    """
    if not csv_path.exists():
        print(f"❌ 파일이 존재하지 않습니다: {csv_path}")
        return []
    
    df = pd.read_csv(csv_path)
    df = df.fillna("")
    
    documents = []
    for _, row in df.iterrows():
        # content를 page_content로
        content = row.get('text', '')

        # 빈 content 필터링 추가
        if not content or len(content.strip()) < 10:
            continue
        
        # 나머지를 metadata로
        metadata = {
            'id': row.get('id', ''),
            'law_name': row.get('src_title', ''),
            'article': row.get('article', ''),
            'priority': float(row.get('priority', 99)),
            'effective_date': row.get('effective_date', ''),
            'source_file': row.get('source', ''),
        }
        
        # 빈 값 제거
        metadata = {k: v for k, v in metadata.items() if v}
        
        documents.append(Document(page_content=content, metadata=metadata))

    return documents


def upload_to_pinecone(documents, index_name, embedding_model):
    """
    Document 리스트를 Pinecone 인덱스에 업로드
    
    Args:
        documents: LangChain Document 리스트
        index_name: Pinecone 인덱스명
        embedding_model: 임베딩 모델
    
    Returns:
        PineconeVectorStore: 생성된 벡터스토어
    """
    if not documents:
        print(f"⚠️ 업로드할 문서가 없습니다.")
        return None
    
    # metadata의 priority를 명시적으로 float로 변환
    for doc in documents:
        if 'priority' in doc.metadata:
            doc.metadata['priority'] = float(doc.metadata['priority'])
    
    print(f"\n{'='*70}")
    print(f"📤 Pinecone 업로드 시작")
    print(f"   인덱스: {index_name}")
    print(f"   문서 수: {len(documents)}")
    print(f"{'='*70}")
    
    vectorstore = PineconeVectorStore.from_documents(
        documents=documents,
        embedding=embedding_model,
        index_name=index_name
    )
    
    print(f"✅ 업로드 완료: {index_name}")
    return vectorstore


print("✅ 헬퍼 함수 정의 완료")

✅ 헬퍼 함수 정의 완료


## 6️⃣ Law 인덱스 업로드 (법률)

In [52]:
# Law CSV 로드
print("📖 Law 데이터 로드 중...")
law_documents = load_csv_to_documents(LAW_CSV)

if law_documents:
    print(f"✅ {len(law_documents)}개 문서 로드 완료")
    
    print("\n🔍 디버깅:")
    for doc in law_documents[:5]:
        print(f"Length: {len(doc.page_content)}, Content: {doc.page_content[:100]}")
    
    # 샘플 확인
    print("\n📄 샘플 문서:")
    print(f"   Content: {law_documents[0].page_content[:100]}...")
    print(f"   Metadata: {law_documents[0].metadata}")
else:
    print("❌ 문서 로드 실패")

📖 Law 데이터 로드 중...
✅ 33개 문서 로드 완료

🔍 디버깅:
Length: 1452, Content: 민법
[시행 2026. 1. 1.] [법률 제20432호, 2024. 9. 20., 일부개정]
법무부(법무심의관실) 02-2110-3164
제3편 채권
제1장 총칙
제2절 채권의 
Length: 1489, Content: ②임차인이 유익비를 지출한 경우에는 임대인은 임대차종료시에 그 가액의 증가가 현존한 때에 한하여 임차인의 지출한 금액이나 그 증가액을 상환하여야 한다. 이 경우에 법원은 임대인의 
Length: 1364, Content: 제637조(임차인의 파산과 해지통고) ①임차인이 파산선고를 받은 경우에는 임대차기간의 약정이 있는 때에도 임대인 또는 파산관재인은 제635조의 규정에 의하여 계약해지의 통고를 할 
Length: 1396, Content: 제646조(임차인의 부속물매수청구권) ①건물 기타 공작물의 임차인이 그 사용의 편익을 위하여 임대인의 동의를 얻어 이에 부속한 물건이 있는 때에는 임대차의 종료시에 임대인에 대하여
Length: 536, Content: 제2조(상속권 상실 선고에 관한 적용례) 제1004조의2의 개정규정은 2024년 4월 25일 이후 상속이 개시되는 경우로서 같은 개정규정 시행 전에 같은 조 제1항 또는 제3항 각

📄 샘플 문서:
   Content: 민법
[시행 2026. 1. 1.] [법률 제20432호, 2024. 9. 20., 일부개정]
법무부(법무심의관실) 02-2110-3164
제3편 채권
제1장 총칙
제2절 채권의 ...
   Metadata: {'id': '2cbc1359-4c34-44a3-b98c-a94c327accc7', 'law_name': '민법(법률) - 임대차편', 'article': '제390조', 'priority': 4.0, 'effective_date': '2026-01-02', 'source_file': 'c:\\project-AI2\\실프로젝트\\data\\raw\\law\\

In [53]:
# CSV 컬럼명 확인
df = pd.read_csv(LAW_CSV)
print(df.columns.tolist())
print(df.head(1))

['id', 'chunk_id', 'text', 'src_title', 'article', 'title', 'category', 'priority', 'effective_date', 'source', 'parsed_at']
                                     id   chunk_id  \
0  2cbc1359-4c34-44a3-b98c-a94c327accc7  chunk_000   

                                                text      src_title article  \
0  민법\n[시행 2026. 1. 1.] [법률 제20432호, 2024. 9. 20....  민법(법률) - 임대차편   제390조   

    title             category  priority effective_date  \
0  법무심의관실  수선_원상회복, 분쟁해결, 계약해지         4     2026-01-02   

                                              source  \
0  c:\project-AI2\실프로젝트\data\raw\law\민법(법률)(제2043...   

                    parsed_at  
0  2026-01-19T17:28:38.102011  


In [54]:
# Law 인덱스에 업로드
if law_documents:
    law_vectorstore = upload_to_pinecone(
        documents=law_documents,
        index_name=LAW_INDEX_NAME,
        embedding_model=embedding
    )
else:
    print("⚠️ 업로드 건너뜀: 문서 없음")


📤 Pinecone 업로드 시작
   인덱스: housing-law-index
   문서 수: 33
✅ 업로드 완료: housing-law-index


In [55]:
# Law 인덱스 통계 확인
if LAW_INDEX_NAME in existing_indexes:
    law_index = pc.Index(LAW_INDEX_NAME)
    law_stats = law_index.describe_index_stats()
    
    print(f"\n📊 {LAW_INDEX_NAME} 통계:")
    print(f"   - Dimension: {law_stats['dimension']}")
    print(f"   - Total Vectors: {law_stats['total_vector_count']}")
    print(f"   - Index Fullness: {law_stats.get('index_fullness', 'N/A')}")


📊 housing-law-index 통계:
   - Dimension: 4096
   - Total Vectors: 33
   - Index Fullness: 0.0


## 7️⃣ Rule 인덱스 업로드 (시행령, 시행규칙)

In [56]:
# Rule CSV 로드
print("📖 Rule 데이터 로드 중...")
rule_documents = load_csv_to_documents(RULE_CSV)

if rule_documents:
    print(f"✅ {len(rule_documents)}개 문서 로드 완료")
    
    # 샘플 확인
    print("\n📄 샘플 문서:")
    print(f"   Content: {rule_documents[0].page_content[:100]}...")
    print(f"   Metadata: {rule_documents[0].metadata}")
else:
    print("❌ 문서 로드 실패")

📖 Rule 데이터 로드 중...
✅ 4개 문서 로드 완료

📄 샘플 문서:
   Content: 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙
[시행 2021. 6. 10.] [대법원규칙 제2986호, 2021. 5. 27., 타법개정]
법원행정처(부동산등기과) ...
   Metadata: {'id': '59c1c89f-3181-4871-88c2-dfd2bd496ab5', 'law_name': '확정일자 대법원규칙', 'article': '제1조', 'priority': 3.0, 'effective_date': '2021-06-10', 'source_file': 'c:\\project-AI2\\실프로젝트\\data\\raw\\rule\\주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙(대법원규칙)(제02986호)(20210610).docx'}


In [57]:
# Rule 인덱스에 업로드
if rule_documents:
    rule_vectorstore = upload_to_pinecone(
        documents=rule_documents,
        index_name=RULE_INDEX_NAME,
        embedding_model=embedding
    )
else:
    print("⚠️ 업로드 건너뜀: 문서 없음")


📤 Pinecone 업로드 시작
   인덱스: housing-rule-index
   문서 수: 4
✅ 업로드 완료: housing-rule-index


In [58]:
# Rule 인덱스 통계 확인
if RULE_INDEX_NAME in existing_indexes:
    rule_index = pc.Index(RULE_INDEX_NAME)
    rule_stats = rule_index.describe_index_stats()
    
    print(f"\n📊 {RULE_INDEX_NAME} 통계:")
    print(f"   - Dimension: {rule_stats['dimension']}")
    print(f"   - Total Vectors: {rule_stats['total_vector_count']}")
    print(f"   - Index Fullness: {rule_stats.get('index_fullness', 'N/A')}")


📊 housing-rule-index 통계:
   - Dimension: 4096
   - Total Vectors: 4
   - Index Fullness: 0.0


## 8️⃣ Case 인덱스 업로드 (판례, 사례)

In [59]:
# Case CSV 로드
print("📖 Case 데이터 로드 중...")
case_documents = load_csv_to_documents(CASE_CSV)

if case_documents:
    print(f"✅ {len(case_documents)}개 문서 로드 완료")
    
    # 샘플 확인
    print("\n📄 샘플 문서:")
    print(f"   Content: {case_documents[0].page_content[:100]}...")
    print(f"   Metadata: {case_documents[0].metadata}")
else:
    print("❌ 문서 로드 실패")

📖 Case 데이터 로드 중...
✅ 160개 문서 로드 완료

📄 샘플 문서:
   Content: 전세피해지원 프로그램 및
전세피해 상담 사례집
  국토부 전세사기피해자 및 HUG 전세피해확인서 
1. 국토부 전세사기피해자 선정기준 05
2. HUG 전세피해확인서 발급기준 07...
   Metadata: {'id': '11b18b5c-0e7b-4ba4-8662-336ad7054ad7', 'law_name': '상담사례 - 2025전세피해지원사례집', 'priority': 7.0, 'source_file': 'c:\\project-AI2\\실프로젝트\\data\\raw\\case\\2025전세피해지원사례집.pdf'}


In [60]:
# Case 인덱스에 업로드
if case_documents:
    case_vectorstore = upload_to_pinecone(
        documents=case_documents,
        index_name=CASE_INDEX_NAME,
        embedding_model=embedding
    )
else:
    print("⚠️ 업로드 건너뜀: 문서 없음")


📤 Pinecone 업로드 시작
   인덱스: housing-case-index
   문서 수: 160
✅ 업로드 완료: housing-case-index


In [61]:
# Case 인덱스 통계 확인
if CASE_INDEX_NAME in existing_indexes:
    case_index = pc.Index(CASE_INDEX_NAME)
    case_stats = case_index.describe_index_stats()
    
    print(f"\n📊 {CASE_INDEX_NAME} 통계:")
    print(f"   - Dimension: {case_stats['dimension']}")
    print(f"   - Total Vectors: {case_stats['total_vector_count']}")
    print(f"   - Index Fullness: {case_stats.get('index_fullness', 'N/A')}")


📊 housing-case-index 통계:
   - Dimension: 4096
   - Total Vectors: 160
   - Index Fullness: 0.0


## 9️⃣ 전체 업로드 결과 요약

In [62]:
print("\n" + "="*70)
print("📊 전체 업로드 결과 요약")
print("="*70)

# 각 인덱스 통계 가져오기
for idx_name in [LAW_INDEX_NAME, RULE_INDEX_NAME, CASE_INDEX_NAME]:
    if idx_name in [idx.name for idx in pc.list_indexes()]:
        idx = pc.Index(idx_name)
        stats = idx.describe_index_stats()
        print(f"\n📌 {idx_name}:")
        print(f"   ✅ 총 벡터 수: {stats['total_vector_count']}")
        print(f"   ✅ 차원: {stats['dimension']}")
    else:
        print(f"\n❌ {idx_name}: 인덱스 없음")

print("\n" + "="*70)
print("✅ 모든 업로드 작업 완료!")
print("="*70)


📊 전체 업로드 결과 요약

📌 housing-law-index:
   ✅ 총 벡터 수: 33
   ✅ 차원: 4096

📌 housing-rule-index:
   ✅ 총 벡터 수: 4
   ✅ 차원: 4096

📌 housing-case-index:
   ✅ 총 벡터 수: 160
   ✅ 차원: 4096

✅ 모든 업로드 작업 완료!


## 🔍 검색 테스트 (옵션)

In [63]:
# 테스트 쿼리
test_queries = [
    ("보증금을 못 받으면 어떻게 하나요?", LAW_INDEX_NAME),
    ("확정일자는 어떻게 받나요?", RULE_INDEX_NAME),
    ("임대인이 사망했는데 보증금은 누구에게 받나요?", CASE_INDEX_NAME)
]

print("\n" + "="*70)
print("🔍 검색 테스트")
print("="*70)

for query, index_name in test_queries:
    print(f"\n{'='*70}")
    print(f"📌 질의: {query}")
    print(f"📍 인덱스: {index_name}")
    print(f"{'-'*70}")
    
    # 해당 인덱스의 vectorstore 생성
    vs = PineconeVectorStore(
        index_name=index_name,
        embedding=embedding
    )
    
    # 검색
    results = vs.similarity_search(query, k=3)
    
    for i, doc in enumerate(results, 1):
        meta = doc.metadata
        print(f"\n[{i}] 📌 Priority {meta.get('priority', '?')}")
        print(f"    📄 {meta.get('law_name', 'Unknown')} - {meta.get('article', 'Unknown')}")
        print(f"    📝 {doc.page_content[:100]}...")


🔍 검색 테스트

📌 질의: 보증금을 못 받으면 어떻게 하나요?
📍 인덱스: housing-law-index
----------------------------------------------------------------------

[1] 📌 Priority 1.0
    📄 주택임대차보호법(법률) - 제3조
    📝 ② 제3조제1항ㆍ제2항 또는 제3항의 대항요건(對抗要件)과 임대차계약증서(제3조제2항 및 제3항의 경우에는 법인과 임대인 사이의 임대차계약증서를 말한다)상의 확정일자(確定日字)를 ...

[2] 📌 Priority 1.0
    📄 주택임대차보호법 시행령(대통령령)(제35947호)(20260102) - 제7조
    📝 ② 「국민기초생활 보장법」에 따른 수급자 등 법무부령으로 정하는 사람에 대해서는 제1항에 따른 수수료를 면제할 수 있다.
[본조신설 2013. 12. 30.]
[종전 제7조는 제1...

[3] 📌 Priority 1.0
    📄 주택임대차보호법(법률) - 제3조
    📝 1. 임차인이 제3조제1항ㆍ제2항 또는 제3항의 대항요건을 상실한 경우
2. 제3조의3제5항에 따른 임차권등기가 말소된 경우
3. 「민법」 제621조에 따른 임대차등기가 말소된 경...

📌 질의: 확정일자는 어떻게 받나요?
📍 인덱스: housing-rule-index
----------------------------------------------------------------------

[1] 📌 Priority 3.0
    📄 확정일자 대법원규칙 - 제3조의2
    📝 1. 해당 주택의 임대인ㆍ임차인
2. 해당 주택의 소유자
3. 해당 주택 또는 그 대지의 등기기록에 기록된 권리자 중 대법원예규로 정하는 자
4. 법 제3조의2제7항에 따라 우선변...

[2] 📌 Priority 3.0
    📄 확정일자 대법원규칙 - 제1조
    📝 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙
[시행 2021. 6. 10.] [대법

In [ ]:
import pandas as pd
import os

# 파일 경로 리스트
file_paths = [
    r'C:\project-AI2\실프로젝트\data\processed\csv\rule.csv',
    r'C:\project-AI2\실프로젝트\data\processed\csv\case.csv',
    r'C:\project-AI2\실프로젝트\data\processed\csv\law.csv'
]

for path in file_paths:
    if os.path.exists(path):
        # 안쪽 따옴표를 작은따옴표(')로 수정하여 SyntaxError 해결
        print(f"\n{'='*30}")
        print(f"파일 확인: {os.path.basename(path)}")
        print(f"{'='*30}")
        
        df = pd.read_csv(path)
        
        # 중복 제거 후 출력
        result = df[['src_title', 'priority']].drop_duplicates().sort_values(by='priority')
        print(result)
    else:
        print(f"\n경로를 찾을 수 없음: {path}")

SyntaxError: f-string: expecting '}' (1298138757.py, line 15)

## ✅ 완료

**다음 단계:**
1. `rag_test.ipynb` 생성하여 통합 검색 테스트
2. 키워드 카테고리 기반 인덱스 라우팅 구현
3. Priority 기반 재정렬 로직 추가